In [1]:
import pandas as pd
from cells.Cell_2g import Cell_2g
from tqdm import tqdm
from datetime import datetime
from geopy.distance import geodesic
import math
import multiprocess as mp
import numpy as np
import pickle

In [9]:
## Import cell level data in below format
## not linked with .mdb to keep it general for all vendors
### Site,Cell,bcch,bsic,band,azimuth,long,lat

In [2]:
# In production user will be asked for file
cell_data=pd.read_csv("D:/Office backup/tools/python/rsi clash/Cells-2g-2022-08-29.csv")

In [3]:
# get site data separately  
site_data=pd.read_csv("D:/Office backup/tools/python/rsi clash/Sites-2022-08-29.csv")

In [28]:
def get_tier(curr_quad_dict):
    ''' Takes a dictionary of current quads and appends tier in value'''
    key=[]
    val=[]    
    for i in range(6):# we have 6 quads
        curr_quad_dict={k: v for k, v in dict_for_tier.items() if v[2]==i} # filter dictionary for current quad
        curr_quad_tier=sorted(curr_quad_dict.items(),key=lambda kv:kv[1][0])[:8]# Get first 7 tiers 
        for i,item in enumerate(curr_quad_tier):
            key.append(item[0]) # Append current site as key
            val.append((item[1][0],item[1][1],i)) # Append current values
    return dict(zip(key,val))

In [30]:
# We are calculating angle and distance between two sites
# We will store this all in a dictionary 

def make_tiers(site_df1,total_sitedf):
    dist_angle_tier_dict={}
    for i,site1 in site_df1.iterrows():
        dict_for_tier={}
        for j,site2 in site_data.iterrows(): 
            if(site1.site==site2.site):
                continue
            dist=geodesic((site1.Latitude, site1.Longitude), (site2.Latitude, site2.Longitude)).meters
            if(dist<20000):# skip sites more than 20 KM away take this distance from user in production envoirment
                angle=(math.atan2(site2.Longitude - site1.Longitude, site2.Latitude - site1.Latitude)*180/math.pi+360)%360
                quad=math.floor(angle/60)
                dict_for_tier[site2.site]=(dist,angle,quad)# Add current site in temporary dictionary
        dist_angle_tier_dict[site1.site]=get_tier(dict_for_tier) # Calculate tiers of the site once we have tiers we can filter them easily


IndentationError: expected an indented block (618134510.py, line 6)

In [12]:
start=datetime.now()
def main():
    manager=mp.Manager()
    p1=mp.Process(target=firstprocess,args=())
    p2=mp.Process(target=secondprocess,args=())
    p3=mp.Process(target=thirdprocess,args=())
    p4=mp.Process(target=fourthprocess,args=())
    p5=mp.Process(target=fifthprocess,args=())
    p6=mp.Process(target=sixthprocess,args=())
    ####################### Start them all #################
    p1.start()
    p2.start()
    p3.start()
    p4.start()
    p5.start()
    p6.start()
    ############### Join them all ##############################
    p1.join()
    p2.join()
    p3.join()
    p4.join()
    p5.join()
    p6.join()
def firstprocess():
    import site_dist_utils as u
    u.make_site_tiers(0)
def secondprocess():
    import site_dist_utils as u
    u.make_site_tiers(1)
def thirdprocess():
    import site_dist_utils as u
    u.make_site_tiers(2)
def fourthprocess():
    import site_dist_utils as u
    u.make_site_tiers(3)
def fifthprocess():
    import site_dist_utils as u
    u.make_site_tiers(4)
def sixthprocess():
    import site_dist_utils as u
    u.make_site_tiers(5)
if __name__=="__main__":
    main()
print(datetime.now()-start)

0:34:28.531950


In [3]:
site_dict={}
for i in range(6): ## We created 6 processes
    file = open(f"D:/Office backup/tools/python/rsi clash/final_dict_part{i}", 'rb')
    site_dict={**site_dict ,** pickle.load(file)}
    

In [5]:
len(site_dict)

3594

In [13]:
sorted_for_current_site=dict(sorted(temp.items(), key=lambda x: x[1][2]))

In [15]:
# get cells which need to be checked

In [4]:
cell_list=[]
for i,rows in tqdm(cell_data.iterrows()):
    c=Cell_2g(rows.site, rows.cell,rows.Latitude,rows.Longitude,rows.BCCH,rows.Band,rows.Azimuth,rows.BSIC)
    cell_list.append(c)

7818it [00:01, 5867.18it/s]


In [5]:
def generate_candid_cell_list(cella,site_dict,tier=5):
    '''  returns cell list for cella with in allowed tier'''
    curr_site=cella.site
    candid_sites=site_dict[str(int(curr_site))] # take sites from dictionary
    all_sites=dict(filter(lambda x: x[1][2]<=tier, candid_sites.items())) # take sites within check tier
    return [cell for cell in cell_list if str(int(cell.site)) in all_sites]

In [6]:
result=[]
max_tier=10
for cell in tqdm(cell_list):
    candid_cell_list=generate_candid_cell_list(cell,site_dict,max_tier)
    curr_clashes=[res for res in candid_cell_list if(cell.isBcchBsicClash(res))]
    # Adjust tiering
    for clashed_cell in curr_clashes:
        site_site_rel=site_dict[str(int(cell.site))][str(int(clashed_cell.site))]
        angle1=site_site_rel[1]-cell.azimuth
        angle2=180+site_site_rel[1]-clashed_cell.azimuth
        adj=math.cos(np.radians(angle1))+math.cos(np.radians(angle2))
        prio=2+site_site_rel[2]-adj
        if(prio<=max_tier):
            result.append([cell.cid,cell.bcch,cell.bsic,clashed_cell.cid,site_site_rel[0],prio])
            

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7818/7818 [00:44<00:00, 175.61it/s]


In [7]:
pd.DataFrame(result,columns=('cell1','bcch','bsic','cell2','dist','prirority (0-10)')).to_csv('BCCH_BSIC Clash.csv',index=False)

In [20]:
pd.DataFrame(result).to_csv("bcch_bsic.csv")

In [11]:
result_df=pd.DataFrame(result,columns=('cell1','bcch','bsic','cell2','dist','prio'))   
result_df.sort_values(by='prio',inplace=True)


In [14]:
src=int(result_df.iloc[0]['cell1'])
tgt=int(result_df.iloc[0]['cell2'])


In [17]:
src

18513

In [19]:
cell_data

,site,Band,cell,Latitude,Longitude,Azimuth,BCCH,NCC,BCC,BSIC
0,1000,900,10001,24.848440,46.825190,20,47,1,1,11
1,1000,900,10002,24.848440,46.825190,110,45,4,2,42
2,1000,900,10003,24.848440,46.825190,280,4,1,0,10
3,100,900,1001,24.586524,46.780306,0,42,6,1,61
4,1001,900,10011,24.686750,46.674999,120,44,1,0,10
...,...,...,...,...,...,...,...,...,...,...
7813,648,900,6483,24.790787,46.652120,200,3,1,3,13
7814,6430,900,64301,24.070170,47.491840,0,1,0,1,1
7815,6430,900,64302,24.070170,47.491840,110,46,0,7,7
7816,6430,900,64303,24.070170,47.491840,220,40,0,1,1


In [20]:
cell_data[cell_data['cell']==18513]

,site,Band,cell,Latitude,Longitude,Azimuth,BCCH,NCC,BCC,BSIC
2220,1851,900,18513,24.670538,46.58564,330,48,2,0,20


In [26]:
cell_data[cell_data['cell']==src][['Latitude','Longitude']].iloc[0][0]

24.670538

In [18]:
site_dict['5529']['4029']

(3879.876978360458, 228.3821223913267, 5)

In [21]:
result=[]
max_tier=2
for cell in tqdm(cell_list):
    candid_cell_list=generate_candid_cell_list(cell,site_dict,max_tier)
    curr_clashes=[res for res in candid_cell_list if(cell.isBcchClash(res))]
    # Adjust tiering
    for clashed_cell in curr_clashes:
        site_site_rel=site_dict[str(int(cell.site))][str(int(clashed_cell.site))]
        angle1=site_site_rel[1]-cell.azimuth
        angle2=180+site_site_rel[1]-clashed_cell.azimuth
        adj=math.cos(np.radians(angle1))+math.cos(np.radians(angle2))
        prio=site_site_rel[2]-adj
        if(prio<=max_tier):
            result.append([cell.cid,cell.bcch,clashed_cell.cid,site_site_rel[0],prio])
            

100%|█████████████████████████████████████████████████████████████████████████████| 7818/7818 [00:43<00:00, 178.97it/s]


In [22]:
pd.DataFrame(result).to_csv("bcch.csv")